In [1]:
import fileinput
import string
from collections import defaultdict, Counter
from collections.abc import Callable
from itertools import filterfalse
import random

https://www.nytimes.com/games/wordle/index.html

In [2]:
def inc_exc(words: list[str], 
            letter_groups: dict[str, set[str]], 
            include: str, 
            exclude: str) -> list[str]:
    """Filters words based on the characters in include and exclude.
    
    If include and exclude are empty, inc_exc returns the same 
    list of words, albeit a copy of the original list with duplicate words removed.
    
    Args:
        words: A list of words.
        letter_groups: A mapping between a character in a word and 
            a subset from words that contain that character.
        include: A string of characters that must be contained in all the output words.
        exclude: A string of characters that must _not_ be contained in any output words.
        
    Returns:
        A list of filtered words where each word contains all the characters in include, but 
        does not have the characters listed in exclude.
    """
    include, exclude = set(include), set(exclude)
    possible = set(words)
    
    for ch in include:
        possible &= letter_groups[ch]

    print(f'Words that include {include}: {len(possible):,}')
    for ch in exclude:
        possible -= letter_groups[ch]

    print(f'Words that include {include}, but exclude {exclude}: {len(possible):,}')
    
    return list(possible)

def group_by_letter(words: list[str], char_set: str=string.ascii_lowercase) -> dict[str, set[str]]:
    """Groups words into sets based on what character is contained in the word.
    
    Args:
        words: A list of words.
        char_set: A string containing all the characters used to compose words.
        
    Returns:
        A mapping between each character in char_set and 
        a subset of words that contain that character.
    """
    letters = defaultdict(set)
    for ch in char_set:
        for word in words:
            if ch in word:
                letters[ch].add(word)
    return letters

class Possible(object):
    def __init__(self, iterable: list[str]):
        print(f'Sorting values')
        iterable.sort()
        self.values = iterable
    
    def distinct_letters_only(self):
        self.filter(lambda word: len(set(word)) == len(word))
        
    def filter(self, condition: Callable[[str], bool]):
        self.values = list(filter(condition, self.values))
        
    def __repr__(self):
        return f'Possible({self.values})'
    
    def __len__(self):
        return len(self.values)
        

In [3]:
words = [word.strip() for word in fileinput.input('words_alpha.txt')]
print(f'All words: {len(words):,}')

five_letter_words = [word for word in words if len(word) == 5]
five_letters_grouped = group_by_letter(five_letter_words)
five = lambda inc, exc: inc_exc(five_letter_words, five_letters_grouped, inc, exc)

print(f'Five-letter words: {len(five_letter_words):,}')

All words: 370,105
Five-letter words: 15,920


## Guesses

### Method 1: Uniformally random word

In [4]:
distinct_letters = list(filter(lambda word: len(set(word)) == len(word), five_letter_words))

In [5]:
random.choice(distinct_letters)

'perky'

### Method 2: Pick words with letters that occur the most

The end goal is to eliminate as many words as possible.  For the first guess, if we pick a word that has letters that occur the most, we can see if any of those letters can be discarded.  Because we used the most frequently used letters, we theoretically eliminate more words per letter using these letters.

In [6]:
hist = Counter()
for word in five_letter_words:
    for ch in word:
        hist[ch] += 1

In [7]:
hist.most_common(10)

[('a', 8393),
 ('e', 7802),
 ('s', 6537),
 ('o', 5219),
 ('r', 5144),
 ('i', 5067),
 ('l', 4247),
 ('t', 4189),
 ('n', 4044),
 ('u', 3361)]

In [9]:
possible = five('iltn', '')
possible

Words that include {'t', 'n', 'l', 'i'}: 15
Words that include {'t', 'n', 'l', 'i'}, but exclude set(): 15


['altin',
 'clint',
 'intel',
 'intil',
 'nilot',
 'glint',
 'until',
 'flint',
 'inlet',
 'unlit',
 'latin',
 'lints',
 'intnl',
 'linty',
 'linet']

## Filter words

In [10]:
print(f'Words with five letters: {len(five_letter_words):,}')

Words with five letters: 15,920


In [17]:
words = Possible(five('na', 'glitmedpwsr'))

words.filter(lambda word: word[3] == 'n')
words.filter(lambda word: word[0] != 'a')
words.filter(lambda word: word[1] == 'a')
words.filter(lambda word: word[0] == 'c' and word[4] == 'y')
# words.distinct_letters_only()

print(len(words))
words

Words that include {'a', 'n'}: 1,731
Words that include {'a', 'n'}, but exclude {'s', 'i', 'd', 'w', 't', 'l', 'g', 'p', 'r', 'm', 'e'}: 102
Sorting values
1


Possible(['canny'])

## Save results

In [24]:
import sqlite3
from collections import namedtuple
import itertools

Result = namedtuple('Result', ['wordle', 'date', 'words', 'regular', 'high_contrast'])
def println(*objects, **kw):
    print(*objects, end='\n\n', **kw)
    
def print_results(results):
    for result in results:
        result = Result(*result)

        hi = result.high_contrast.splitlines()
        header = hi[0]
        words = result.words.splitlines()

        print(header)
        for line, word in zip(itertools.islice(hi, 2, None), words):
            print(f'{line} {word}')
        print()
    
con = sqlite3.connect('wordle.db')
cur = con.cursor()

In [25]:
print_results(cur.execute('''SELECT * FROM results ORDER BY rowid DESC''').fetchall()[:5])

Wordle 323 5/6*
⬜⬜⬜🟧⬜ glint
🟦⬜⬜🟧⬜ amend
⬜🟧⬜🟧⬜ pawns
🟧🟧⬜🟧🟧 carny
🟧🟧🟧🟧🟧 canny

Wordle 322 3/6*
⬜⬜🟦⬜🟦 roils
⬜🟧🟦🟦⬜ piste
🟧🟧🟧🟧🟧 midst

Wordle 321 5/6*
⬜🟦⬜⬜🟦 selva
🟦⬜⬜🟦🟦 armed
🟦🟦⬜🟦⬜ dewax
⬜🟧🟧⬜🟧 cadie
🟧🟧🟧🟧🟧 badge

Wordle 320 6/6*
⬜🟦🟦⬜🟦 arose
⬜🟧⬜🟧🟧 coder
⬜🟧⬜🟧🟧 tower
⬜🟧⬜🟧🟧 boxer
🟧🟧⬜🟧🟧 hover
🟧🟧🟧🟧🟧 homer

Wordle 319 2/6*
⬜🟦🟧⬜🟦 unapt
🟧🟧🟧🟧🟧 train



### Create table

Only need to do once.  Uncomment to execute.

In [19]:
# cur.execute('''
# CREATE TABLE results 
# (wordle, date, words, regular, high_contrast)
# ''')
# con.commit()

### Save result

To keep it simple, everything is stored as TEXT.

In [20]:
cur.execute('''
INSERT INTO results VALUES (
'323', '2022-05-08', 
'glint
amend
pawns
carny
canny', 
'Wordle 323 5/6*

⬜⬜⬜🟩⬜
🟨⬜⬜🟩⬜
⬜🟩⬜🟩⬜
🟩🟩⬜🟩🟩
🟩🟩🟩🟩🟩', 
'Wordle 323 5/6*

⬜⬜⬜🟧⬜
🟦⬜⬜🟧⬜
⬜🟧⬜🟧⬜
🟧🟧⬜🟧🟧
🟧🟧🟧🟧🟧'
)''')

con.commit()

### SQL utilities

In [21]:
# cur.execute('''DELETE FROM results WHERE rowid=3''')
# con.commit()

In [22]:
# close connection to database
con.close()